In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df=pd.read_csv('./ori_data/Wimbledon_featured_matches.csv')
df.head(2)

,match_id,player1,player2,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,...,p2_break_pt_won,p1_break_pt_missed,p2_break_pt_missed,p1_distance_run,p2_distance_run,rally_count,speed_mph,serve_width,serve_depth,return_depth
0,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:00,1,1,1,0,0,0,...,0,0,0,6.000,7.840,2,95.0,BC,NCTL,ND
1,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:38,1,1,2,0,0,0,...,0,0,0,5.253,7.094,1,118.0,B,CTL,ND


In [3]:
df.columns

Index(['match_id', 'player1', 'player2', 'elapsed_time', 'set_no', 'game_no',
       'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games', 'p1_score',
       'p2_score', 'server', 'serve_no', 'point_victor', 'p1_points_won',
       'p2_points_won', 'game_victor', 'set_victor', 'p1_ace', 'p2_ace',
       'p1_winner', 'p2_winner', 'winner_shot_type', 'p1_double_fault',
       'p2_double_fault', 'p1_unf_err', 'p2_unf_err', 'p1_net_pt', 'p2_net_pt',
       'p1_net_pt_won', 'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt',
       'p1_break_pt_won', 'p2_break_pt_won', 'p1_break_pt_missed',
       'p2_break_pt_missed', 'p1_distance_run', 'p2_distance_run',
       'rally_count', 'speed_mph', 'serve_width', 'serve_depth',
       'return_depth'],
      dtype='object')

In [23]:
player_1_perf_col=['p1_ace','p1_winner','p1_double_fault','p1_unf_err','p1_net_pt',
'p1_net_pt_won','p1_break_pt','p1_break_pt_won','p1_break_pt_missed','p1_distance_run']
player_2_perf_col=['p2_ace','p2_winner','p2_double_fault','p2_unf_err','p2_net_pt',
'p2_net_pt_won','p2_break_pt','p2_break_pt_won','p2_break_pt_missed','p2_distance_run']

player_1_score_col=['p1_sets','p1_games','p1_score','p1_points_won']
player_2_score_col=['p2_sets','p2_games','p2_score','p2_points_won']

conditions_col=['match_id','player1','player2','elapsed_time','set_no','game_no','point_no']

other_col=['server','serve_no','point_victor','game_victor','set_victor','winner_shot_type',
'rally_count','speed_mph','serve_width','serve_depth','return_depth']
#server可以切换成对玩家的有利/有害
#p1_socre和p1_points_won是高度相关的
#

In [33]:
# preassure=difference between set+game+point
len(player_2_perf_col)

10

In [ ]:
# 每局都会变的指标：
server
#p1_games
#p1_points_won

-->p1_server
p1_ace
p1_winner
p1_double_fault
p1_unf_err
p1_net_pt
p1_distance_run

-->p1_preasure

# 
#p1_net_pt_won
#p1_break_pt
#p1_break_pt_won
#p1_break_pt_missed

-->predict: point_victor

In [60]:
# df[df['p1_net_pt']==1]

In [52]:
len(df['match_id'].unique())

31

## How to define Pressure, or strength
strength 一共用累积的胜率差来衡量
Pressure 用胜率 + 当前的set 当前的game 当前的point的差值的和来衡量

In [78]:
player1_strength=df['p1_points_won']/(df['p1_points_won']+df['p2_points_won'])
player2_strength=df['p2_points_won']/(df['p1_points_won']+df['p2_points_won'])

player1_dist=df['p1_distance_run']/(df['p1_distance_run']+df['p2_distance_run'])
player2_dist=df['p2_distance_run']/(df['p1_distance_run']+df['p2_distance_run'])

df['player1_dist']=player1_dist
df['player2_dist']=player2_dist
df['player1_strength']=player1_strength
df['player2_strength']=player2_strength



In [91]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 在 DataFrame 上拟合并转换数据
df['p1_distance_run_normal'] = scaler.fit_transform(df[['p1_distance_run']])
df['p2_distance_run_normal'] = scaler.fit_transform(df[['p2_distance_run']])

In [85]:
p1_server = [1 if i == 1 else 0 for i in df['server']]
p2_server = [1 if i == 2 else 0 for i in df['server']]
df['p1_server']=p1_server
df['p2_server']=p2_server

In [100]:
p1_victor = [1 if i == 1 else 0 for i in df['point_victor']]
p2_victor = [1 if i == 2 else 0 for i in df['point_victor']]
df['p1_victor']=p1_victor
df['p2_victor']=p2_victor

In [ ]:
preasure=df['p1_sets']+(df['p1_sets']-df['p2_sets'])
        +df['p1_games']+(df['p1_games']-df['p2_games'])
        +df['p1_score']+(df['p1_score']-df['p2_score'])

In [142]:
temp_1=df[['match_id','p1_ace','p1_winner','p1_double_fault','p1_unf_err','p1_net_pt','p1_server','player1_strength','p1_distance_run_normal','player1_dist','p1_victor']]
temp_1.head(5)

,match_id,p1_ace,p1_winner,p1_double_fault,p1_unf_err,p1_net_pt,p1_server,player1_strength,p1_distance_run_normal,player1_dist,p1_victor
0,2023-wimbledon-1301,0,0,0,1,0,1,0.000000,-0.593118,0.433526,0
1,2023-wimbledon-1301,0,0,0,0,0,1,0.500000,-0.648485,0.425447,1
2,2023-wimbledon-1301,0,0,0,1,0,1,0.333333,-0.014995,0.410617,0
3,2023-wimbledon-1301,0,1,0,0,0,1,0.500000,2.750213,0.403254,1
4,2023-wimbledon-1301,1,1,0,0,0,1,0.600000,-0.989725,0.443912,1


In [144]:
temp_2=df[['match_id','p2_ace','p2_winner','p2_double_fault','p2_unf_err','p2_net_pt','p2_server','player2_strength','p2_distance_run_normal','player2_dist','p2_victor']]
# temp_2.head(5)

In [145]:
column_mapping = dict(zip(temp_2.columns, temp_1.columns))
temp_2 = temp_2.rename(columns=column_mapping)
temp_2.head(2)

,match_id,p1_ace,p1_winner,p1_double_fault,p1_unf_err,p1_net_pt,p1_server,player1_strength,p1_distance_run_normal,player1_dist,p1_victor
0,2023-wimbledon-1301,0,0,0,0,0,0,1.0,-0.443108,0.566474,1
1,2023-wimbledon-1301,0,0,0,0,0,0,0.5,-0.497934,0.574553,0


In [146]:
# print(len(temp[(temp['player1_dist']<0.5) & (temp['p1_victor']==1)]))
# print(len(temp[(temp['player1_dist']>0.5) & (temp['p1_victor']==1)]))
df_player12 = temp_1.append(temp_2, ignore_index=True)
df_player12

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28960\2793402930.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_player12 = temp_1.append(temp_2, ignore_index=True)


,match_id,p1_ace,p1_winner,p1_double_fault,p1_unf_err,p1_net_pt,p1_server,player1_strength,p1_distance_run_normal,player1_dist,p1_victor
0,2023-wimbledon-1301,0,0,0,1,0,1,0.000000,-0.593118,0.433526,0
1,2023-wimbledon-1301,0,0,0,0,0,1,0.500000,-0.648485,0.425447,1
2,2023-wimbledon-1301,0,0,0,1,0,1,0.333333,-0.014995,0.410617,0
3,2023-wimbledon-1301,0,1,0,0,0,1,0.500000,2.750213,0.403254,1
4,2023-wimbledon-1301,1,1,0,0,0,1,0.600000,-0.989725,0.443912,1
...,...,...,...,...,...,...,...,...,...,...,...
14563,2023-wimbledon-1701,0,0,0,0,1,0,0.500000,2.726725,0.568930,0
14564,2023-wimbledon-1701,0,0,0,0,0,0,0.498489,1.021317,0.612950,0
14565,2023-wimbledon-1701,0,1,0,0,0,0,0.500000,0.327688,0.627456,1
14566,2023-wimbledon-1701,0,0,0,0,0,0,0.498498,-0.244897,0.571731,0


In [147]:
df_player12.to_csv('player_12.csv')
df.to_csv('cleaned_N01.csv')

# Train NN

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import pandas as pd

df=pd.read_csv('./player_12.csv')

In [98]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# 假设 df 是你的 DataFrame
# 假设 target_column 是你的目标列
# 假设其他列是特征列

# 随机打乱 DataFrame
df_shuffled = shuffle(df, random_state=42)

# 划分训练集、验证集和测试集
train_df, temp_df = train_test_split(df_shuffled, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 定义 PyTorch Dataset 类
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert features to float32 and handle non-numeric values
        features = torch.tensor(self.data.iloc[idx, 8:-1].astype('float32').values, dtype=torch.float32)
        
        # Convert target to float32
        target = torch.tensor(self.data.iloc[idx, -1], dtype=torch.int64)  # Assuming the first column is the target
        
        return features, target

# 创建训练集、验证集和测试集的 Dataset
train_dataset = CustomDataset(train_df)
val_dataset = CustomDataset(val_df)
test_dataset = CustomDataset(test_df)

# 创建 DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [99]:
# for i in val_loader:
#     print(len(i[1]))
train_df

,Unnamed: 0,match_id,p1_ace,p1_winner,p1_double_fault,p1_unf_err,p1_net_pt,p1_server,player1_strength,p1_distance_run_normal,player1_dist,p1_victor
7228,7228,2023-wimbledon-1701,0,0,0,0,0,0,0.494624,-0.401226,0.563435,0
13134,13134,2023-wimbledon-1501,0,0,0,0,0,0,0.472222,0.635845,0.570614,0
8701,8701,2023-wimbledon-1306,0,0,0,0,1,1,0.470000,-0.614567,0.535544,1
9440,9440,2023-wimbledon-1309,0,0,0,1,0,1,0.470270,-0.580099,0.561549,0
623,623,2023-wimbledon-1303,0,0,0,0,0,0,0.398374,0.203061,0.485867,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5494,5494,2023-wimbledon-1408,0,0,0,0,0,0,0.491525,-0.951184,0.568580,0
6168,6168,2023-wimbledon-1502,0,0,0,0,0,1,0.527473,1.012730,0.456731,1
9027,9027,2023-wimbledon-1307,0,0,0,1,1,1,0.474227,0.004613,0.451356,0
11429,11429,2023-wimbledon-1402,0,1,0,0,0,1,0.496241,1.714945,0.491434,1


In [100]:
import torch.nn as nn
import torch.nn.functional as F

# class SimpleNet(nn.Module):
#     def __init__(self):
#         super(SimpleNet, self).__init__()
#         self.fc1 = nn.Linear(9, 100)
#         self.fc2 = nn.Linear(100, 200)
#         self.fc3 = nn.Linear(200, 100)
#         self.fc4 = nn.Linear(100, 50)
#         self.fc5 = nn.Linear(50, 2)
#         self.dropout = nn.Dropout(p=0.2)  # 添加 Dropout 层
#         self.batch_norm1 = nn.BatchNorm1d(100)  # 添加 Batch Normalization 层
#         self.batch_norm2 = nn.BatchNorm1d(200)
#         self.batch_norm3 = nn.BatchNorm1d(100)
#         self.batch_norm4 = nn.BatchNorm1d(50)

#     def forward(self, x):
#         x = x.view(-1, 9)
#         x = F.relu(self.fc1(x))
#         x = self.batch_norm1(x)
#         x = self.dropout(x)
#         x = F.relu(self.fc2(x))
#         x = self.batch_norm2(x)
#         x = self.dropout(x)
#         x = F.relu(self.fc3(x))
#         x = self.batch_norm3(x)
#         x = self.dropout(x)
#         x = F.relu(self.fc4(x))
#         x = self.batch_norm4(x)
#         x = self.dropout(x)
#         x = self.fc5(x)
#         return x
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(3, 2)
        self.fc2 = nn.Linear(2, 2)
        # self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 3)
        x = F.relu(self.fc1(x))
        # x = F.sigmoid(self.fc2(x))
        x = self.fc2(x)
        return x


In [101]:
simplenet = SimpleNet()

In [108]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.00000000001)

In [109]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=3, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=2, bias=True)
)

In [111]:
temp=[]
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            print(output)
            print(inputs)
            temp.append([output,targets])
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            # temp.append(batch)
            # temp.append(targets)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        # print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        # valid_loss, num_correct / num_examples))

train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_loader,val_loader, epochs=1, device=device)

Epoch: 1, Training Loss: nan, Validation Loss: nan, accuracy = 0.50


In [83]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_loader,val_loader, epochs=1, device=device)

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward0>)
tensor([[ 1.0000,  1.0000,  0.5088, -0.2159,  0.5144],
        [ 0.0000,  0.0000,  0.4810, -0.8984,  0.5325],
        [ 0.0000,  1.0000,  0.5285, -0.7843,  0.5110],
        [ 0.0000,  0.0000,  0.5300, -0.6270,  0.5937],
        [ 0.0000,  0.0000,  0.4809,  0.3927,  0.4192],
        [ 0.0000,  0.0000,  0.3784, -0.7438,  0.5480],
   

In [14]:
# temp[0]
for i in train_loader:
    print(i)
    # exit()

[tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5455, -0.9982,
          0.3669],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5714, -0.9415,
          0.5731],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.4919, -0.7451,
          0.4778],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4443,
          0.6057],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.5000, -0.7322,
          0.4530],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5238, -0.9377,
          0.5736],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4663, -0.6045,
          0.5047],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4000, -0.9595,
          0.5561],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4902, -0.9121,
          0.5445],
        [ 1.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.5249, -0.9738,
          0.5376],

In [195]:
# for batch in (val_loader):
    # print(batch)
batch[1]

21